In [237]:
from pyxing.session import *
from pyxing.query import *
from pyxing.real import *
from threading import Thread
import pandas as pd

import threading

import time
from datetime import datetime

import pythoncom
import win32com.client as winAPI
import numpy as np

In [238]:
shcode_list = ['A033780', 'A064350', 'A017670', 'A005490', 'A014680', 'A012450', 'A051900', 'A009150', 'A066570', 'A108320', 'A047810', 'A000660', 'A010140', 'A035420', 'A005380', 'A009830', 'A052690', 'A034020', 'A005070', 'A005420', 'A042700', 'A028050', 'A068270', 'A137310', 'A005930', 'A017960', 'A075580', 'A003490']

In [239]:
for shcode in shcode_list:
    globals()['df_{}'.format(shcode)] = pd.DataFrame(columns=['종목코드','시간','현재가','체결수량', '거래량', '매도누적체결수량', '매수누적체결수량'])

In [240]:
receive_list = ['shcode','chetime','price','cvolume','volume','mdvolume','msvolume']

In [241]:
# login
xasession = XASession()
xasession.login(id='ssuyan',password='tndus1!!',cert='qkrtndus1!!',block=True)
print('연결상태:',xasession.is_connected())

연결상태: True


In [242]:
# query = XAQuery('t1310')
# data = query(shcode='shcode', cnt=900)

# query = XAQuery('t1310')
# data1 = query.call(shcode='shcode', cnt=100).data
# data2 = query.nextcall(
#     cts_time=data1['t1310OutBlock']['시간CTS'][0]
# ).data

# query = XAQuery('t1310')
# data3 = query.call(shcode='shcode', cnt=100).data
# data4 = query.nextcall(
#     shcode='shcode', 
#     cnt=100, 
#     cts_time=data1['t1310OutBlock']['시간CTS'][0]
# ).data

# query = XAQuery('t1310')
# data = query.call(shcode='shcode', cnt=100).next(keypairs={'time':'cts_time'})

In [243]:
import win32com.client
import pythoncom

# XingRealTime(object)

class XingRealTime(object):
    realTime = None

    def __init__(self):
        super().__init__()

        self.realTime = win32com.client.DispatchWithEvents("XA_DataSet.XAReal", RealTimeDataHandler)
        RealTimeDataHandler.realTime = self.realTime
    #def end

    def OnReceiveRealData(self, XAReal):
        shcode = self.realTime.GetFieldData('OutBlock', 'shcode')
        chetime = self.realTime.GetFieldData('OutBlock', 'chetime')
        price = self.realTime.GetFieldData('OutBlock', 'price')
        cvolume = self.realTime.GetFieldData('OutBlock', 'cvolume')
        volume = self.realTime.GetFieldData('OutBlock', 'cvolume')
        mdvolume = self.realTime.GetFieldData('OutBlock', 'mdvolume')
        msvolume = self.realTime.GetFieldData('OutBlock', 'msvolume')
        print("종목;{0},시간;{1}, 현재가;{2}, 체결수량;{3}, 거래량;{4}, 매도누적체결수량;{5}, 매수누적체결수량;{6}".format(shcode, chetime, price, cvolume, volume, mdvolume, msvolume))

    def subscribeRealTimeData(self, shcode):
        self.realTime.LoadFromResFile("C:\\eBEST\\xingAPI\\Res\\S3_.res")
        self.realTime.SetFieldData('InBlock', 'shcode', shcode)
        self.realTime.AdviseRealData()
    #def end

    def add_item(self, shcode_list):
    # 실시간데이터 요청 종목 추가
        self.SetFieldData("InBlock", "shcode", shcode_list)
        self.AdviseRealData()

    def waitEvent(self):
        # Waiting query event
        while RealTimeDataHandler.realTimeState == 0:
            pythoncom.PumpWaitingMessages()
    #def end

# def recieve_dt(self,shcode,shcode_list):
#     xaquery = XAQuery()
#     re_df = self.realTime.ReceivedRealData('S3_',shcode=shcode)
    # re_time = xaquery.set_field_data('t0167',id=re_df[0]['chetime'])


    # dttime = (re_time[0]['dt']+re_time[0]['time']).values[0]
    # re_df = re_df[0][receive_list[1:]].values[0].tolist()

    # df = globals()['df_{}'.format(shcode)]

    # df.loc[i, 'Datetime'] = dttime
    # df.iloc[i,1:] = re_df
# #class end


class RealTimeDataHandler:
    realTimeState = 0
    realTime = None

    def OnReceiveRealData(self, XAReal):
        shcode = self.realTime.GetFieldData('OutBlock', 'shcode')
        chetime = self.realTime.GetFieldData('OutBlock', 'chetime')
        price = self.realTime.GetFieldData('OutBlock', 'price')
        cvolume = self.realTime.GetFieldData('OutBlock', 'cvolume')
        volume = self.realTime.GetFieldData('OutBlock', 'cvolume')
        mdvolume = self.realTime.GetFieldData('OutBlock', 'mdvolume')
        msvolume = self.realTime.GetFieldData('OutBlock', 'msvolume')
        print("종목;{0},시간;{1}, 현재가;{2}, 체결수량;{3}, 거래량;{4}, 매도누적체결수량;{5}, 매수누적체결수량;{6}".format(shcode, chetime, price, cvolume, volume, mdvolume, msvolume))
        
    #def end
# class end


In [244]:
queue = Queue()
xareal = XAReal(queue)
xareal.register_res("S3_.res")
xareal.set_field_data("shcode",shcode)
xareal.advise_real_data()


In [245]:
t2 = Thread(target=XingRealTime, args=(queue,))
t2.start()

while True:
    pythoncom.PumpWaitingMessages()
    xareal.add_item(shcode) 

Exception in thread Thread-15:
Traceback (most recent call last):
  File "c:\Users\tj\anaconda3\envs\best\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "c:\Users\tj\anaconda3\envs\best\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
TypeError: __init__() takes 1 positional argument but 2 were given



AttributeError: 'XAReal' object has no attribute 'add_item'

In [ ]:
# def multithreading_xing(i):
#     start = time.perf_counter()
#     threads=[]

#     for shcode in shcode_list:
#         t = threading.Thread(target=recieve_dt(shcode,i))
#         t.start()
#         threads.append(t)

#     for thread in threads:
#         thread.join()

#     finish = time.perf_counter()
